## APPLICATION TEMPLATE ERA5 - GLOBAL DAILY MEAN TEMPERATURE

package c3s-eqc-toolbox-template (https://github.com/bopen/c3s-eqc-toolbox-template)

In [ ]:
import plotly.express as px
import pandas as pd
import xarray as xr

import cads_toolbox 
import c3s_eqc_toolbox_template

import warnings
warnings.filterwarnings('ignore')

### PROCESSING with Xarray: GLOBAL DAILY MEAN TEMPERATURE 2021-2022 

APPLICATION SUMMARY:

   - Data requests definition
   
   
   - Processing: 
       - chunked download
       - apply tranformation to each chunk
       - chache the result on each chunk
       - merge all the chunks


   - Plot the result

#### REQUEST DEFINITION

For request definition we use **`c3s_eqc_toolbox_template.update_request_date`**.

It allows to generate the requests for a given period `[start, stop]` if stop is `None` then compute the stop month as follows: 
if `current day > switch_month_day`: then `stop_month = current_month - 1`
else `stop_month = current_month - 2` 

Returns the request or the list of requests for the input period.  

In [ ]:
collection_id = 'reanalysis-era5-single-levels'

request = {
    'product_type': 'reanalysis',
    'format': 'grib',
    'time': [
        '06', '18' 
    ],
    'variable': '2m_temperature',
}
start = "2021-06"
stop = None # "2022-06"

In [ ]:
requests = c3s_eqc_toolbox_template.update_request_date(request, start=start, stop=stop, switch_month_day=9)
requests

#### DOWNLOAD and PROCESSING

The processing will be perform as follows:

**Definition of the transformation `f`** to be applied to each single chunk

**Chunk definition**: define how to chunk the request along time dimension (year, month, day): 
e.g. `chunks={"year": 1,"month": 1}`


**Downlaod and processing**: for each chunk in time
    
    - Download chunk
    - Cache the downloaded chunk (optional)
    - Apply the operation `f` to the chunk
    - Cache transformed chunk, `f(chunks)` (optional)

Merge all the transformed chunk

#### Define `f`

In [ ]:
def spatial_daily_mean(ds: xr.Dataset) ->  xr.Dataset:
    ds = c3s_eqc_toolbox_template.statistics.spatial_mean(ds)
    return ds.resample(time="1D").mean("time")

#### Define `chunks`

In [ ]:
chunks={"year": 1,"month": 1}

#### Perform the download and processing

In [ ]:
cads_toolbox.config.USE_CACHE = True

In [ ]:
daily_global_t2m = c3s_eqc_toolbox_template.download_and_transform(
    collection_id, requests, chunks=chunks, f=spatial_daily_mean, open_with="xarray",
)

daily_global_t2m

#### Plot result 

In [ ]:
daily_global_t2m = daily_global_t2m.squeeze()
fig = px.line(
    x=daily_global_t2m["time"],
    y=daily_global_t2m["t2m"] - 273.15, 
)
fig.update_layout(
    xaxis_title="time", 
    yaxis_title="t2m", 
    title="GLOBAL DAILY MEAN TEMPERATURE 2021-2022"
)
fig.show()

In [ ]:
fig.write_image("ERA5_global_daily_mean_temperature_2021-2022.png")

<hr style="border:2px solid gray">

## Example using pandas: DAILY MEAN TEMPERATURE ON ROME 2021-2022

#### Define operation to be applied to each single chunk

In [ ]:
def daily_mean_on_rome(df: pd.DataFrame)-> pd.DataFrame:
    lat, lon = 42, 12.5
    df = df.reset_index()
    df = df[(df.latitude == 42) & (df.longitude == 12.5)]
    return df.groupby(df.valid_time.dt.floor("d")).mean()

#### Downaload and perform the processing

In [ ]:
daily_rome_t2m = c3s_eqc_toolbox_template.download_and_transform(
    collection_id, requests, chunks={"year": 1,"month": 1}, f=daily_mean_on_rome, open_with="pandas")

daily_rome_t2m 

#### Plot result 

In [ ]:
daily_global_t2m = daily_rome_t2m.squeeze()
fig = px.line(daily_rome_t2m  - 273.15, y="t2m")

fig.update_layout(
    xaxis_title="time", 
    yaxis_title="t2m", 
    title="DAILY MEAN TEMPERATURE ON ROME 2021-2022"
)

fig.show()

In [ ]:
fig.write_image("ERA5_daily_mean_temperature_on_rome_2021-2022.png")